In [ ]:
# Load datasets

import os
from dotenv import load_dotenv
load_dotenv() # API Keys are stored in .env file

# Load D-Fire dataset from Google Drive
#!pip install gdown
import gdown 
from zipfile import ZipFile
df_url = "https://drive.google.com/uc?id=19LSrZHYQqJSdKgH8Mtlgg7-i-L3eRhbh"  # smoke = 0, fire = 1
df_destination = "datasets\\D-Fire.zip"
df_directory = "datasets\\D-Fire"

if not os.path.isdir(df_directory):
    if not os.path.isfile(df_destination):
        print("Downloading dataset...", end='\r')
        gdown.download(df_url, df_destination, quiet=False)
        print("D-Fire dataset downloaded")
    else:
        with ZipFile(df_destination, 'r') as zip_ref:
            print("Unpacking..." , end="\r")
            zip_ref.extractall(df_directory)
            print("Files extracted")

# Load FASDD Dataset from Kaggle
#!pip install kagglehub
import kagglehub
fasdd_path = kagglehub.dataset_download("cookiecacheqq/fasdd-cv")

# Load datasets from Roboflow
#!pip install roboflow
from roboflow import Roboflow

roboflow_api_key = os.getenv("RF_API_KEY")
rf = Roboflow(api_key=roboflow_api_key)

# Download roboflow datasets
os.chdir("datasets")
datasets = [
    rf.workspace("data2").project("fire-smoke-detection-ua3dm").version(2), # fire = 0, smoke = 1
    rf.workspace("brad-dwyer").project("wildfire-smoke").version(1), # smoke = 0
]
datasets = [d.download("yolov11") for d in datasets]
print("Roboflow datasets downloaded")
os.chdir("..")

locations = [d.location for d in datasets]

# Standardize label indices: SMOKE = 0, FIRE = 1
# swap indices in dataset 0
import re
import builtins

count = 0
for d in ["train", "valid", "test"]:
    in_path = os.path.join(datasets[0].location, d, "labels")
    out_path = os.path.join(datasets[0].location + "_corrected_labels", d, "labels")
    for f in os.listdir(in_path):
        filename = os.fsdecode(f)
        if not filename.endswith(".txt"):
            continue
        with builtins.open(os.path.join(in_path, filename), 'r') as file:
            o_contents = file.read()
        with builtins.open(os.path.join(out_path, filename), 'w') as file:
            contents = re.sub("0 ", "2 ", o_contents)
            contents = re.sub("1 ", "0 ", contents)
            contents = re.sub("2 ", "1 ", contents)
            file.write(contents)

yaml_data = f"""
path: Combined
train: train\\images
val: valid\\images
test: test\\images

nc: 2
names: ['smoke', 'fire']
"""

with open("data.yaml", 'w') as file:
    file.write(yaml_data)

 14%|██████████▌                                                                | 1.61G/11.5G [10:08<1:01:51, 2.86MB/s]

In [7]:
# Start Tensorflow dashboard
%reload_ext tensorboard
%tensorboard --logdir runs/detect

In [ ]:
# Load pretrained YOLO model for transfer learning
#!pip install ultralytics
from ultralytics import YOLO
import os

# Training options
opts = {
    "data": "data.yaml",
    "batch": 256,
    "val": True,
    "optimizer": "auto",
    "pretrained": True,
    "freeze": 11, # transfer learning
    "imgsz": 224, # larger is better for texture recognition but slow
    "plots": True,
    "workers": 1, # fix for old Nvidia GPU
}

# Train model to establish baseline performance
model = YOLO("yolo11n.pt")
model.train(**opts, epochs=50, name="yolo_v11n_224px")

In [2]:
from ultralytics import YOLO
# Load best-fit model
model = YOLO("runs/detect/yolo_v11n_defaults/weights/best.pt")


In [ ]:
# Prune model
from torch.nn.utils import prune

ratio = 0.25

for name, module in model.named_modules():
    if ".conv" in name:
        print(f"Pruning layer {name}...")
        prune.l1_unstructured(module, name='weight', amount=ratio)
        prune.remove(module, 'weight')
print("Pruning done")

# Fine-tune after pruning0
model.train(**opts | {"freeze": False, "epochs":5, "name":"yolo_v11n_pruned"}) #override optimizer and lr
model.save(f'pruned_model.pt')

In [5]:
# Test on random inputs from dataset
import random
import os

random.seed()
num_imgs = 5

img_dir = "datasets/Combined" + "/test/images"
random_imgs = [random.choice(os.listdir(img_dir)) for x in range(num_imgs)]
results = model.predict([os.path.join(img_dir, im) for im in random_imgs])

for result in results:
    #boxes = result.boxes
    result.show()


0: 640x640 (no detections), 21.9ms
1: 640x640 (no detections), 21.9ms
2: 640x640 1 smoke, 21.9ms
3: 640x640 1 smoke, 21.9ms
4: 640x640 1 smoke, 21.9ms
5: 640x640 2 smokes, 6 fires, 21.9ms
6: 640x640 (no detections), 21.9ms
7: 640x640 2 smokes, 4 fires, 21.9ms
8: 640x640 4 fires, 21.9ms
9: 640x640 (no detections), 21.9ms
Speed: 5.5ms preprocess, 21.9ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)
